<a href="https://colab.research.google.com/github/unography/EIP3-Assignments/blob/master/EIP3_Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/Drive')

!cp /Drive/My\ Drive/alice.txt .

Drive already mounted at /Drive; to attempt to forcibly remount, call drive.mount("/Drive", force_remount=True).


In [0]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import re

In [0]:
filename = "alice.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [0]:
def remove_punc(text):
  return re.sub('[^A-Za-z0-9. \n]+', '', text)

In [0]:
raw_text = remove_punc(raw_text)


In [0]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [0]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  137077
Total Vocab:  31


In [0]:
sentence_split = [x.split('.') for x in raw_text.split('\n')]
print(len(sentence_split))
sentences = []
for sent in sentence_split:
  for each_sent in sent:
    if each_sent:
      each_sent = each_sent.strip()
      each_sent += "."
      sentences.append(each_sent)
  

3337


In [0]:
sentences[0], len(sentences)

('alices adventures in wonderland.', 2849)

In [0]:
dataX = []
dataY = []
seq_length = 30
for sent in sentences:
  if len(sent) <= seq_length:
    dataX.append([char_to_int[char] for char in sent[:-1]])
    dataY.append(char_to_int[sent[-1]])
  else:
    for i in range(0, len(sent) - seq_length, 1):
      seq_in = sent[i:i + seq_length]
      seq_out = sent[i + seq_length]
      dataX.append([char_to_int[char] for char in seq_in])
      dataY.append(char_to_int[seq_out])

seq_length = 30
dataX = pad_sequences(dataX, maxlen=seq_length)
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

In [0]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [159]:
X.shape

(59825, 30, 1)

In [0]:
model = Sequential()
model.add(Dropout(0.1))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
checkpoint = ModelCheckpoint('model.h5', monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [162]:
model.fit(X, y, epochs=100, batch_size=128, callbacks=callbacks_list)

Epoch 1/100
59825/59825 [==============================] - 44s 728us/step - loss: 2.8881

Epoch 00001: loss improved from inf to 2.88806, saving model to model.h5
Epoch 2/100
59825/59825 [==============================] - 38s 643us/step - loss: 2.7332

Epoch 00002: loss improved from 2.88806 to 2.73320, saving model to model.h5
Epoch 3/100
59825/59825 [==============================] - 38s 643us/step - loss: 2.6383

Epoch 00003: loss improved from 2.73320 to 2.63833, saving model to model.h5
Epoch 4/100
59825/59825 [==============================] - 39s 649us/step - loss: 2.5416

Epoch 00004: loss improved from 2.63833 to 2.54162, saving model to model.h5
Epoch 5/100
59825/59825 [==============================] - 40s 673us/step - loss: 2.4666

Epoch 00005: loss improved from 2.54162 to 2.46665, saving model to model.h5
Epoch 6/100
59825/59825 [==============================] - 39s 658us/step - loss: 2.3829

Epoch 00006: loss improved from 2.46665 to 2.38287, saving model to model.h5
Ep

In [0]:
model = load_model('model.h5')

In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [164]:
# pick a random seed
import sys
start = numpy.random.randint(0, len(dataX)-1)
start = 1231
print(dataX[start])
pattern = list(dataX[start])
for value in pattern:
  print(value, int_to_char[value])
print(''.join([int_to_char[value] for value in pattern]))
# print "Seed:"
# print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
# print "\nDone."

[24 19  1 12  9 22 23  9 16 10  1  5 10 24  9 22  1 23 25  7 12  1  5  1
 10  5 16 16  1  5]
24 t
19 o
1  
12 h
9 e
22 r
23 s
9 e
16 l
10 f
1  
5 a
10 f
24 t
9 e
22 r
1  
23 s
25 u
7 c
12 h
1  
5 a
1  
10 f
5 a
16 l
16 l
1  
5 a
to herself after such a fall a
nd sanes soees asd.whe qther iad gooedt whetes anl harder what sueste.bnice asd whe muryrds oeanled oe leave tueer hand interieted.tatulmlar and tuundd ay the sfppee uolether srlstamlsl anlce hopves wo tee sfe her hnyn mtoerie tucste tei swer tuermemes anl about forre taid alice as see no knce a doovowson of noteer deoied hirself.sutnswat all as m all sanpirg at vhe cooksttte ier iead triselinartty anice asd whe mooanes wo go cetteisly and tuierlnl whe sacbie and tatee ofar toeery anl toutai sut